<a href="https://colab.research.google.com/github/DonErnesto/amld2021-unsupervised/blob/master/notebooks/exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Exercise: Demonstration of several algorithms on the Pen Digits dataset

Exercises using the Pen Digits Dataset: https://www.dbs.ifi.lmu.de/research/outlier-evaluation/DAMI/literature/PenDigits/PenDigits_v01.html

The data is already present in .csv format (load the data below).
This data set was created by recording the writing pattern of digits on a digital writing pad. The digit "4" is downsampled to only 20 instances (instead of ~1000 for the other points), making it an outlier. 

Note that (unlike MNIST) the features are not pixel values, but are x,y subsampled coordinate pairs, for a total of 8 pairs. 

This dataset is small and simple: it has only numeric features and no NaN's. 

# Package installing and data import

In [ ]:
if 'google.colab' in str(get_ipython()):
    print('Running on CoLab, need to get data and install libraries..')
    data_path = './'
    # Now only load the required files...
    !curl -O https://raw.githubusercontent.com/DonErnesto/amld2021-unsupervised/master/notebooks/outlierutils.py
    !curl -O https://raw.githubusercontent.com/DonErnesto/amld2021-unsupervised/master/data/x_pendigits.csv
    !curl -O https://raw.githubusercontent.com/DonErnesto/amld2021-unsupervised/master/data/y_pendigits.csv
    !pip install --upgrade pyod
else:
    print('Not running on CoLab, data and libraries are already present')
    data_path = '../data'


## Imports 

In [ ]:
# standard library imports
import os

import seaborn as sns
import sklearn 
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np

In [ ]:
from outlierutils import plot_top_N, plot_outlier_scores # For easy plotting and evaluation

## Data loading

In [ ]:
from sklearn.preprocessing import StandardScaler
x_pen_raw = pd.read_csv(os.path.join(data_path, 'x_pendigits.csv'))
y_pen = pd.read_csv(os.path.join(data_path, 'y_pendigits.csv'))['outlier']

# Scale and put again into a DataFrame
sc = StandardScaler()
x_pen = pd.DataFrame(data=sc.fit_transform(x_pen_raw))

In [ ]:
print(f'Number of points: {len(y_pen)}')
print(f'Number of positives: {y_pen.sum()} ({y_pen.mean():.3%})')


## Sample data

In [ ]:
non_outlier = y_pen[y_pen == 0]
outlier = y_pen[y_pen == 1]  

fig, axs = plt.subplots(1, 5, figsize=(15, 3))
axs = axs.ravel()
for ax, i in enumerate([15, 4, 1, 8, 1234]):
    xcoor = x_pen_raw.iloc[non_outlier.index[i], :].values.reshape([8,2])
    axs[ax].plot(xcoor[:,0], xcoor[:,1],'ro-')
    axs[ax].axis('equal')
    axs[ax].axis('off')
fig.suptitle('Non outlier [any digit except 4s]')

fig, axs = plt.subplots(1, 5, figsize=(15, 3))
axs = axs.ravel()
for ax, i in enumerate(range(5)):
    xcoor = x_pen_raw.iloc[outlier.index[i], :].values.reshape([8,2])
    axs[ax].plot(xcoor[:,0], xcoor[:,1],'ro-')
    axs[ax].axis('equal')
    axs[ax].axis('off')
fig.suptitle('Outlier [4s digits]');

The data contains the x, y coordinates of the pen strokes when writing the digits. The non-outliers show numbers from 0-9 (except 4), and the outliers represent the digit 4.

## Demo: Usage of plotting functions

See examples how to plot the conditional scores and the top-N ranking below

In [ ]:
# example with random data
N_top = 1000
y_true_, scores_ = np.random.choice([0, 1], N_top), np.random.uniform(size=N_top)
results = plot_outlier_scores(y_true=y_true_, 
                              scores=scores_, 
                              bw_adjust=0.5, 
                              title='Example.')

In [ ]:
# The next plot shows the true labels of the N points with the highest outlier scores.
# More yellow is better!

results = plot_top_N(y_true=y_true_, scores=scores_, N=50)

## Data visualization

t-SNE of large datasets may take a long time to compute. The next piece of code will downsample the negatives, while retaining all positives. 

In [ ]:
from sklearn.manifold import TSNE

N_downsample = 3000
assert x_pen.index.equals(y_pen.index), 'Error, indexes differ. Reset them to continue'
x_downsampled = pd.concat((x_pen[y_pen==0].sample(N_downsample - int(y_pen.sum()), random_state=1),
                           x_pen[y_pen==1]), 
                          axis=0).sample(frac=1, random_state=1)
y_downsampled = y_pen[x_downsampled.index]

In [ ]:
type(x_downsampled)

#### Q 0. 
Reduce the dimensionality with T-SNE, and visualize the positive and negative class in a scatter plot. 
What do you observe?

**Hint**: To get help for a function or class, run `?<object>`  

In [ ]:
?TSNE

In [ ]:
MAX_N_TSNE = 4000 #Avoid overly long computation times with TSNE. Values < 4000 recommended 
neg = y_downsampled == 0
pos = y_downsampled == 1

assert len(x_downsampled) <= MAX_N_TSNE, f'Using a dataset with more than {MAX_N_TSNE} points is not recommended'
X_2D = TSNE().fit_transform(x_downsampled) # transform to 2-D space for plotting


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 8))
ax.scatter(X_2D[pos, 0], X_2D[pos, 1], c=[[0.8, 0.4, 0.4],], marker='x', s=120, label='Positive')
ax.scatter(X_2D[neg, 0], X_2D[neg, 1], c=[[0.2, 0.3, 0.9],], marker='o', s=10, label='Negative')

plt.axis('off')
plt.legend()
plt.show() 

In [ ]:
del x_downsampled, y_downsampled # To avoid using the wrong data later

## Mahalanobis Distance

Using `EmpiricalCovariance`, or `MinCovDet` (a robust estimator), do a `.fit()` to fit the covariance matrix. 
Determine the distance with `.mahalanobis()` and use this as outlier score. Get the Area Under the ROC-Curve, PR-curve and Precision@100 using `plot_outlier_scores` and `plot_top_N`

In [ ]:
from sklearn.covariance import MinCovDet, EmpiricalCovariance
cov_ = EmpiricalCovariance().fit(x_pen)

In [ ]:
dist = cov_.mahalanobis(x_pen)
#plot_outlier_scores(y_pen, dist)
plot_top_N(y_pen, dist, 1000)

## GMM

Using `GaussianMixture`, with a reasonable value for n_components and `covariance_type=full`, do a `.fit()` and `.score_samples()` to get the log-probability of each sample. The negative log-probability will be the outlier score.


In [ ]:
from sklearn.mixture import GaussianMixture

# gmm = GaussianMixture(n_components=xxxx, covariance_type='full', random_state=1) # try also spherical
# gmm.fit(x_pen)


#### Q 1. 
Which algorithm performed better, GMM or Mahalonobis? Why do you think?

## KNN algorithm


#### Q 2.1
With the scikit-learn NearestNeighbors class, determine the probability of the nearest neighbour of a point being an outlier, conditional on the class membership of that point. If the nearest neighbour of the outliers is often an outlier, we can conclude that the outliers form one (or more than one) cluster. Compare this result with the observations on the t-SNE visualization
 
The code provided in the next cell calculates the indices of the nearest neighbour of all points.


In [ ]:
from sklearn.neighbors import NearestNeighbors

# clf_nn = NearestNeighbors(n_neighbors=10)
# clf_nn.fit(x_pen)
# # the function .kneighbors() returns one array with distances and one array with indexes of neighbours:
# distances, indices = clf_nn.kneighbors(x_pen)
# # for each point, look at the index of the closest point (the 0-th is the point itself, so we look for the point at index 1):
# indices_nearest_neighbor = indices[:, 1]

# # find out how many of the nearest neighbors of each outlier are outliers themselves.
# # In other words, find out the conditional label of the nearest neighbors of each point, conditioned on the point being an outlier.
# # use the numpy mask y_pen==1 as needed to consider just the outliers


#### Q 2.2
Use pyod's KNN class to detect the outliers. Use `method=largest` (the default) and guess a reasonable value for `n_neighbors` based on the insights from t-SNE and the previous question. For the scores, you can use the `.decision_scores_` attributes of the fitted KNN model. This is simply a vector of distances.

Plot the conditional score curves and the top-100 results, using `plot_outlier_scores` and `plot_top_N`. 



In [ ]:
from pyod.models.knn import KNN
# clf_knn = KNN(method='median', n_neighbors=xxx)
# clf_knn.fit(x_pen)
## get the prediction label and outlier scores of the training data
# scores_knn = clf_knn.decision_scores_

#### Q 3.

Vary `n_neighbors`, how does it affect:
- AUC-ROC (area under the curve (AUC) in the Receiver Operating Characteristic (ROC) (the higher the better, 0.5 for a random classifier)
- precision@100, which is the fraction of outliers among the highest 100 scoring points

**HINT**: 
1. Use clf.decision_scores_ as in the previous step
2. Run `roc_auc_score?` to see the docs for calculating AUC-ROC
3. Use the provided function `calc_precision_at_100` to calculate the precision@100 for a scores vector `scores`

In [ ]:
from pyod.models.knn import KNN
from sklearn.metrics import roc_auc_score

def calc_precision_at_100(scores):
    y_pen[np.argsort(scores)][::-1][:100].mean()




## LOF 

Note that the LOF algorithm compares the "reachability density" of a point to its k nearest neighbours, compared to that same density of its nearest neighbours.

Also here, we will use a pyod class. Therefore, we can use the same method- and attribute names. 


#### Q 4.
Considering the concept that underlies LOF, and the results from the t-SNE/Nearest neighbour analysis, do you expect LOF to do better than KNN with n_neighbours=10? Why?

#### Q 5.
Plot the scoring curves for a few values of n_neighbours. What is a good value?


In [ ]:
from pyod.models.lof import LOF
# lof = LOF(n_neighbors=n_neighbours, contamination=0.01)




## Isolation Forest


#### Q 6. 
What disadvantage of Isolation Forest do you see compared to the previous algorithms?

Run an `IsolationForest`analysis with a reasonable set of parameters


In [ ]:
from pyod.models.iforest import IForest
#ifo = IForest(behaviour='new',n_estimators=10, max_samples=512)
#ifo.fit(xxx)

In [ ]:
# as an alternative you can use sklearn
# from sklearn.ensemble import IsolationForest

#### (optional question)
Compare the scatter in AUC scores by running ten times the Isolation Forest, with different `random_state`s, for `n_estimators`=100

## PCA reconstruction error

#### Q 7.
What number of components would you estimate to be suffificient? How may it be determined?

#### Q 8.
Determine the Euclidean reconstruction error, by first transforming the data, and then applying the inverse transform. What scores do you get?

**Use the sci-kit learn implementation, rather than pyod's PCA class. This one seems not to be implemented well**


In [ ]:
from sklearn.decomposition import PCA
from pyod.models.pca import PCA as pyod_PCA
# pca = PCA(n_components=...)
# pca_tf = ... 
# x_pen_recon = ...

In [ ]:
# pca_recon = ((x_pen - x_pen_recon)**2).mean(axis=1)


## Autoencoder reconstruction error

Run the autoencoder with a bottleneck size that worked well for PCA. Run for ~10-15 epochs and look at AUC score. 
Many different configurations (number of hidden layers, number of neurons) may be used, but pick one. 



#### Q 9. 
What output activation do you think will work best? 



In [ ]:
from pyod.models.auto_encoder import AutoEncoder

clf = AutoEncoder(
    hidden_neurons=[xx, xx, xx], # Choose size here!
    hidden_activation='elu',
    output_activation='xx', # Choose an activation ('linear', 'sigmoid', 'relu', 'elu' are some possibilities)
    optimizer='adam',
    epochs=15,
    batch_size=16,
    dropout_rate=0.0, #may not be needed here
    l2_regularizer=0.0,
    validation_size=0.1,
    preprocessing=False, #NB: this uses sklearn's StandardScaler
    verbose=1,
    random_state=1,
)


In [ ]:
# clf.fit(x_pen)


#### Q 10.

- Which algorithm performed best?
- Can it be reasonably "tuned" without having the labels available?
